In [1]:
!pip install pycaret pandas shap


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Train Pm2.5 (-7days)

In [ ]:
import pandas as pd
from pycaret.regression import *
from datetime import timedelta
import numpy as np

# โหลดไฟล์ CSV
file_path = "D:\project-term\pm_2.5\export-jsps013-1h-7days.csv"
df = pd.read_csv(file_path)

# แปลง datetime เป็นชนิด datetime
df["datetime"] = pd.to_datetime(df["datetime"], dayfirst=True, errors="coerce")

# แปลง 'pm_2_5' เป็นตัวเลข
df["pm_2_5"] = pd.to_numeric(df["pm_2_5"], errors="coerce")

# สร้างสำเนา DataFrame เพื่อใช้ทำความสะอาดข้อมูล
df_cleaned = df.copy()

# ฟังก์ชันสำหรับจัดการ outlier และ NaN ด้วย Hybrid Imputation
def hybrid_imputation(series):
    # คำนวณ IQR
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1

    # กำหนดขอบเขตของ outlier
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # คำนวณค่ากลางจากข้อมูลที่ไม่ใช่ outlier
    non_outlier_values = series[(series >= lower_bound) & (series <= upper_bound)]
    median_value = non_outlier_values.median() if not non_outlier_values.empty else series.median()

    # แทนที่ outlier และ NaN ด้วยค่าข้างเคียง + ค่ากลาง
    series = series.mask((series < lower_bound) | (series > upper_bound), np.nan)
    series = series.interpolate(method="linear")  # เติมค่าที่ขาดหายด้วย Interpolation
    series = series.fillna(median_value)  # ถ้ายังมี NaN อยู่ให้เติมด้วยค่ากลาง

    return series

# ใช้ Hybrid Imputation สำหรับคอลัมน์ 'pm_2_5'
df_cleaned["pm_2_5"] = hybrid_imputation(df_cleaned["pm_2_5"])

# สร้างฟีเจอร์ย้อนหลัง (Lag Features)
for lag in [1, 3, 6, 24]:
    df_cleaned[f"pm_2_5_lag_{lag}"] = df_cleaned["pm_2_5"].shift(lag)

# เพิ่มฟีเจอร์เชิงเวลา (Hour และ Day of Week)
df_cleaned["hour"] = df_cleaned["datetime"].dt.hour
df_cleaned["dayofweek"] = df_cleaned["datetime"].dt.dayofweek

# ฟังก์ชันเพิ่ม Fourier Terms สำหรับ Weekly Seasonality
def add_fourier_terms(df, period=24 * 7, n_harmonics=4):
    for k in range(1, n_harmonics + 1):
        df[f"sin_{k}"] = np.sin(2 * np.pi * k * df["hour"] / period)
        df[f"cos_{k}"] = np.cos(2 * np.pi * k * df["hour"] / period)
    return df

# เพิ่ม Fourier Terms ลงใน DataFrame
df_cleaned = add_fourier_terms(df_cleaned)

# ตั้งค่า PyCaret และแบ่ง train 80% test 20%
exp = setup(df_cleaned, target="pm_2_5", train_size=0.8, session_id=42, normalize=True, feature_selection=True)

# ปรับแต่งโมเดล Random Forest Regressor
model = create_model("rf")

final_model = finalize_model(model)
save_model(final_model, "D:/project-term/modelPM/model_PM255")

# ---- สร้างข้อมูลสำหรับพยากรณ์ล่วงหน้า 7 วัน (168 ชั่วโมง) ----
last_date = df_cleaned["datetime"].max()
future_dates = [last_date + timedelta(hours=i) for i in range(1, 169)]

# ใช้ค่าเฉลี่ยล่าสุดของความชื้นและอุณหภูมิ
latest_humidity = df_cleaned["humidity"].iloc[-24:].mean()
latest_temperature = df_cleaned["temperature"].iloc[-24:].mean()
latest_pm25 = df_cleaned["pm_2_5"].iloc[-1]

# สร้าง DataFrame สำหรับพยากรณ์
future_data = pd.DataFrame({
    "datetime": future_dates,
    "humidity": [latest_humidity] * len(future_dates),
    "temperature": [latest_temperature] * len(future_dates),
    "pm_2_5_lag_1": [latest_pm25] * len(future_dates),
    "pm_2_5_lag_3": [latest_pm25] * len(future_dates),
    "pm_2_5_lag_6": [latest_pm25] * len(future_dates),
    "pm_2_5_lag_24": [latest_pm25] * len(future_dates)
})

# เพิ่มฟีเจอร์เชิงเวลาให้ future_data
future_data["hour"] = future_data["datetime"].dt.hour
future_data["dayofweek"] = future_data["datetime"].dt.dayofweek

# เพิ่ม Fourier Terms ให้ future_data
future_data = add_fourier_terms(future_data)

# โหลดโมเดลที่บันทึกไว้
final_model = load_model("D:/project-term/modelPM/model_PM25")

# Recursive forecasting (พยากรณ์แบบก้าวหน้า)
for i in range(len(future_data)):
    pred = predict_model(final_model, data=future_data.iloc[i:i+1])
    predicted_value = pred.iloc[0]["prediction_label"]
    if i + 1 < len(future_data):
        future_data.at[i + 1, "pm_2_5_lag_1"] = predicted_value
    if i + 3 < len(future_data):
        future_data.at[i + 3, "pm_2_5_lag_3"] = predicted_value
    if i + 6 < len(future_data):
        future_data.at[i + 6, "pm_2_5_lag_6"] = predicted_value
    if i + 24 < len(future_data):
        future_data.at[i + 24, "pm_2_5_lag_24"] = predicted_value

# ทำนายค่า PM 2.5
predictions = predict_model(final_model, data=future_data)

# หาชื่อคอลัมน์การทำนายที่ถูกต้อง
prediction_column = [col for col in predictions.columns if "predict" in col.lower()]
if prediction_column:
    prediction_column = prediction_column[0]
    print(predictions[["datetime", prediction_column]])
else:
    print("ไม่พบคอลัมน์การทำนาย ลองแสดงผลทั้งหมด:")
    print(predictions.head())

future_data.to_csv("D:/project-term/pm_2.5/future_pm_datax.csv", index=False)

# บันทึกผลลัพธ์เป็นไฟล์ CSV โดยเลือกเฉพาะ datetime และ prediction_label
predictions[["datetime", "prediction_label"]].to_csv("D:/project-term/pm_2.5/predicted_pm25.csv", index=False)

print("บันทึกผลลัพธ์การพยากรณ์สำเร็จ! ✅")



Graph actual_pm_2_5 compare PM 2.5 (7 Days)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# โหลดไฟล์เก่า (สำหรับการเปรียบเทียบค่าจริง)
file_path_old = r"D:\project-term\pm_2.5\export-jsps013-1h.csv"
df_old = pd.read_csv(file_path_old)

# ฟังก์ชันสำหรับจัดการ outlier และ NaN ด้วย Hybrid Imputation
def hybrid_imputation(series):
    # คำนวณ IQR
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1

    # กำหนดขอบเขตของ outlier
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # คำนวณค่ากลางจากข้อมูลที่ไม่ใช่ outlier
    non_outlier_values = series[(series >= lower_bound) & (series <= upper_bound)]
    median_value = non_outlier_values.median() if not non_outlier_values.empty else series.median()

    # แทนที่ outlier ด้วยค่าเฉลี่ยของค่าข้างเคียงแทนค่ามัธยฐานคงที่
    series = series.mask((series < lower_bound) | (series > upper_bound), np.nan)
    series = series.interpolate(method="linear")  # เติมค่าที่ขาดหายด้วย Interpolation
    series = series.fillna(median_value)  # ถ้ายังมี NaN อยู่ให้เติมด้วยค่ากลาง

    return series


# เช็คและแปลง datetime ของไฟล์เก่า
df_old["datetime"] = pd.to_datetime(df_old["datetime"], dayfirst=True, errors="coerce")
df_old["pm_2_5"] = pd.to_numeric(df_old["pm_2_5"], errors="coerce")

# ใช้ Hybrid Imputation สำหรับคอลัมน์ 'pm_2_5' ในไฟล์เก่า
df_old["pm_2_5"] = hybrid_imputation(df_old["pm_2_5"])

# ดึงค่าจริงจากไฟล์เก่า (168 ชั่วโมงสุดท้าย)
actual_values_old = df_old['pm_2_5'].iloc[-168:].reset_index(drop=True)



# ---- เปรียบเทียบและพล็อตกราฟ ----
# เพิ่มคอลัมน์ 'actual_pm_2_5' ใน predictions DataFrame
predictions['actual_pm_2_5'] = actual_values_old

# พล็อตกราฟเปรียบเทียบ
plt.figure(figsize=(20, 10))  # ตั้งค่าขนาดกราฟ

# พล็อตเส้นกราฟ
plt.plot(predictions['datetime'], predictions['actual_pm_2_5'], label='Actual PM 2.5 ', marker='x', markersize=8, linewidth=2, color='blue') #(จากไฟล์เก่า)
plt.plot(predictions['datetime'], predictions['prediction_label'], label='Predicted PM 2.5 ', marker='x', markersize=8, linewidth=2, color='orange') #(ค่าทำนาย)

# เพิ่มเส้นกริด
plt.grid(True, linestyle='--', alpha=0.7)

# ปรับแต่งแกน X และ Y
plt.xlabel('Datetime', fontsize=14)
plt.ylabel('PM 2.5', fontsize=14)
plt.xticks(rotation=45, fontsize=12)  # ปรับ rotation และขนาด font ของแกน X
plt.yticks(fontsize=12)  # ปรับขนาด font ของแกน Y

# เพิ่ม title และ legend
plt.title('actual_pm_2_5 compare PM 2.5 (7 Days)', fontsize=16, fontweight='bold')
plt.legend(fontsize=12, loc='upper left')

# ปรับ layout เพื่อไม่ให้ข้อมูลถูกตัด
plt.tight_layout()

# แสดงกราฟ
plt.show()

Check test Accuracy

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error,mean_absolute_error

# แบ่งข้อมูลเป็นชุดฝึกและชุดทดสอบ
X = df_cleaned.drop(columns=["pm_2_5", "datetime"])  # ฟีเจอร์
y = df_cleaned["pm_2_5"]  # เป้าหมาย (target)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# สร้างและฝึกโมเดล
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# ทำนายค่าบนชุดฝึกและชุดทดสอบ
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# ประเมินประสิทธิภาพด้วย MSE, R², และ MAPE
test_mse = mean_squared_error(y_test, y_test_pred)

test_r2 = r2_score(y_test, y_test_pred)

test_mape = mean_absolute_percentage_error(y_test, y_test_pred)

# แสดงผลลัพธ์
print(f"Test MSE: {test_mse}")
print(f"Test R²: {test_r2}")
print(f"Test MAPE: {test_mape}")


PM2.5 Future next 7 Days

In [ ]:
import pandas as pd
from pycaret.regression import *
from datetime import timedelta
import numpy as np

# โหลดไฟล์ CSV
file_path = "D:/project-term/pm_2.5/export-jsps013-1h.csv"
df = pd.read_csv(file_path)

# แปลง datetime เป็นชนิด datetime
df["datetime"] = pd.to_datetime(df["datetime"], dayfirst=True, errors="coerce")

# แปลง 'pm_2_5' เป็นตัวเลข
df["pm_2_5"] = pd.to_numeric(df["pm_2_5"], errors="coerce")

# สร้างสำเนา DataFrame เพื่อใช้ทำความสะอาดข้อมูล
df_cleaned = df.copy()

# ฟังก์ชันสำหรับจัดการ outlier และ NaN ด้วย Hybrid Imputation
def hybrid_imputation(series):
    # คำนวณ IQR
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1

    # กำหนดขอบเขตของ outlier
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # คำนวณค่ากลางจากข้อมูลที่ไม่ใช่ outlier
    non_outlier_values = series[(series >= lower_bound) & (series <= upper_bound)]
    median_value = non_outlier_values.median() if not non_outlier_values.empty else series.median()

    # แทนที่ outlier และ NaN ด้วยค่าข้างเคียง + ค่ากลาง
    series = series.mask((series < lower_bound) | (series > upper_bound), np.nan)
    series = series.interpolate(method="linear")  # เติมค่าที่ขาดหายด้วย Interpolation
    series = series.fillna(median_value)  # ถ้ายังมี NaN อยู่ให้เติมด้วยค่ากลาง

    return series

# ใช้ Hybrid Imputation สำหรับคอลัมน์ 'pm_2_5'
df_cleaned["pm_2_5"] = hybrid_imputation(df_cleaned["pm_2_5"])

# สร้างฟีเจอร์ย้อนหลัง (Lag Features)
for lag in [1, 3, 6, 24]:
    df_cleaned[f"pm_2_5_lag_{lag}"] = df_cleaned["pm_2_5"].shift(lag)

# เพิ่มฟีเจอร์เชิงเวลา (Hour และ Day of Week)
df_cleaned["hour"] = df_cleaned["datetime"].dt.hour
df_cleaned["dayofweek"] = df_cleaned["datetime"].dt.dayofweek

# ฟังก์ชันเพิ่ม Fourier Terms สำหรับ Weekly Seasonality
def add_fourier_terms(df, period=24 * 7, n_harmonics=4):
    for k in range(1, n_harmonics + 1):
        df[f"sin_{k}"] = np.sin(2 * np.pi * k * df["hour"] / period)
        df[f"cos_{k}"] = np.cos(2 * np.pi * k * df["hour"] / period)
    return df

# เพิ่ม Fourier Terms ลงใน DataFrame
df_cleaned = add_fourier_terms(df_cleaned)

# ตั้งค่า PyCaret และแบ่ง train 80% test 20%
exp = setup(df_cleaned, target="pm_2_5", train_size=0.8, session_id=42, normalize=True, feature_selection=True)

# ปรับแต่งโมเดล Random Forest Regressor
model = create_model("rf")

final_model = finalize_model(model)
save_model(final_model, "D:/project-term/modelPM/model_PM25")

# ---- สร้างข้อมูลสำหรับพยากรณ์ล่วงหน้า 7 วัน (168 ชั่วโมง) ----
last_date = df_cleaned["datetime"].max()
future_dates = [last_date + timedelta(hours=i) for i in range(1, 169)]

# ใช้ค่าเฉลี่ยล่าสุดของความชื้นและอุณหภูมิ
latest_humidity = df_cleaned["humidity"].iloc[-24:].mean()
latest_temperature = df_cleaned["temperature"].iloc[-24:].mean()
latest_pm25 = df_cleaned["pm_2_5"].iloc[-1]

# สร้าง DataFrame สำหรับพยากรณ์
future_data = pd.DataFrame({
    "datetime": future_dates,
    "humidity": [latest_humidity] * len(future_dates),
    "temperature": [latest_temperature] * len(future_dates),
    "pm_2_5_lag_1": [latest_pm25] * len(future_dates),
    "pm_2_5_lag_3": [latest_pm25] * len(future_dates),
    "pm_2_5_lag_6": [latest_pm25] * len(future_dates),
    "pm_2_5_lag_24": [latest_pm25] * len(future_dates)
})

# เพิ่มฟีเจอร์เชิงเวลาให้ future_data
future_data["hour"] = future_data["datetime"].dt.hour
future_data["dayofweek"] = future_data["datetime"].dt.dayofweek

# เพิ่ม Fourier Terms ให้ future_data
future_data = add_fourier_terms(future_data)

# โหลดโมเดลที่บันทึกไว้
final_model = load_model("D:/project-term/modelPM/model_PM25")

# Recursive forecasting (พยากรณ์แบบก้าวหน้า)
for i in range(len(future_data)):
    pred = predict_model(final_model, data=future_data.iloc[i:i+1])
    predicted_value = pred.iloc[0]["prediction_label"]
    if i + 1 < len(future_data):
        future_data.at[i + 1, "pm_2_5_lag_1"] = predicted_value
    if i + 3 < len(future_data):
        future_data.at[i + 3, "pm_2_5_lag_3"] = predicted_value
    if i + 6 < len(future_data):
        future_data.at[i + 6, "pm_2_5_lag_6"] = predicted_value
    if i + 24 < len(future_data):
        future_data.at[i + 24, "pm_2_5_lag_24"] = predicted_value

# ทำนายค่า PM 2.5
predictions = predict_model(final_model, data=future_data)

# หาชื่อคอลัมน์การทำนายที่ถูกต้อง
prediction_column = [col for col in predictions.columns if "predict" in col.lower()]
if prediction_column:
    prediction_column = prediction_column[0]
    print(predictions[["datetime", prediction_column]])
else:
    print("ไม่พบคอลัมน์การทำนาย ลองแสดงผลทั้งหมด:")
    print(predictions.head())

future_data.to_csv("D:/project-term/pm_2.5/future_pm_data.csv", index=False)

# บันทึกผลลัพธ์เป็นไฟล์ CSV โดยเลือกเฉพาะ datetime และ prediction_label
predictions[["datetime", "prediction_label"]].to_csv("D:/project-term/pm_2.5/predicted_pm25.csv", index=False)

print("บันทึกผลลัพธ์การพยากรณ์สำเร็จ! ✅")



,Description,Value
0,Session id,42
1,Target,pm_2_5
2,Target type,Regression
3,Original data shape,"(22954, 18)"
4,Transformed data shape,"(22954, 4)"
5,Transformed train set shape,"(18363, 4)"
6,Transformed test set shape,"(4591, 4)"
7,Numeric features,16
8,Date features,1
9,Rows with missing values,0.1%


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,3.1992,21.5067,4.6375,0.7802,0.3270,0.3244
1,3.1819,20.6813,4.5477,0.7856,0.3340,0.3412
2,3.2192,22.0313,4.6937,0.7849,0.3229,0.3090
3,3.0821,20.3975,4.5164,0.8041,0.3273,0.3234
4,3.0986,20.0165,4.4740,0.8046,0.3134,0.3073
5,3.2126,20.9109,4.5728,0.7956,0.3184,0.3098
6,3.2271,21.8941,4.6791,0.7818,0.3280,0.3276
7,3.1464,20.9690,4.5792,0.7953,0.3255,0.3151
8,3.1851,20.4120,4.5180,0.8007,0.3217,0.3204


Transformation Pipeline and Model Successfully Saved
Transformation Pipeline and Model Successfully Loaded


               datetime  prediction_label
0   2025-02-11 00:00:00          9.997434
1   2025-02-11 01:00:00          9.255073
2   2025-02-11 02:00:00          8.965721
3   2025-02-11 03:00:00          9.013110
4   2025-02-11 04:00:00          8.682686
..                  ...               ...
163 2025-02-17 19:00:00          6.985869
164 2025-02-17 20:00:00          7.434701
165 2025-02-17 21:00:00          7.170258
166 2025-02-17 22:00:00          6.631188
167 2025-02-17 23:00:00          7.278471

[168 rows x 2 columns]
บันทึกผลลัพธ์การพยากรณ์สำเร็จ! ✅


Humidity next 7 days

In [ ]:
import pandas as pd
from pycaret.regression import *
from datetime import timedelta
import numpy as np

# โหลดไฟล์ CSV
file_path = "D:\project-term\pm_2.5\export-jsps013-1h.csv"
df = pd.read_csv(file_path)

# แปลง datetime เป็นชนิด datetime
df["datetime"] = pd.to_datetime(df["datetime"], dayfirst=True, errors="coerce")

# แปลง 'humidity' เป็นตัวเลข
df["humidity"] = pd.to_numeric(df["humidity"], errors="coerce")

# ลบคอลัมน์ 'pm_2_5' และ 'temperature'
for col in ["pm_2_5", "temperature"]:
    if col in df.columns:
        df.drop(columns=[col], inplace=True)

# ฟังก์ชัน Hybrid Imputation
def hybrid_imputation(series):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    non_outlier_values = series[(series >= lower_bound) & (series <= upper_bound)]
    median_value = non_outlier_values.median() if not non_outlier_values.empty else series.median()

    series = series.mask((series < lower_bound) | (series > upper_bound), np.nan)
    series = series.interpolate(method="linear")  
    series = series.fillna(median_value)  

    return series

# ใช้ Hybrid Imputation สำหรับ 'humidity'
df["humidity"] = hybrid_imputation(df["humidity"])

# สร้างฟีเจอร์ย้อนหลังของ humidity
for lag in [1, 3, 6, 24]:
    df[f"humidity_lag_{lag}"] = df["humidity"].shift(lag)

# ลบแถวที่มีค่า NaN หลังจากสร้าง Lag
df.dropna(inplace=True)

# ตั้งค่า PyCaret และแบ่ง train 80% test 20%
exp = setup(df, target="humidity", train_size=0.8, session_id=42, normalize=True, feature_selection=True)


# สร้างโมเดล Random Forest Regressor
model_humidity = create_model("rf")

final_model = finalize_model(model_humidity)
save_model(final_model, "D:/project-term/modelPM/model_humidity")

# ---- สร้างข้อมูลสำหรับพยากรณ์ล่วงหน้า 7 วัน (168 ชั่วโมง) ----
last_date = df["datetime"].max()
future_dates = [last_date + timedelta(hours=i) for i in range(1, 169)]

# ใช้ค่าล่าสุดของ humidity
latest_humidity = df["humidity"].iloc[-1]

future_data = pd.DataFrame({
    "datetime": future_dates,
    "humidity_lag_1": [latest_humidity] * len(future_dates),
    "humidity_lag_3": [latest_humidity] * len(future_dates),
    "humidity_lag_6": [latest_humidity] * len(future_dates),
    "humidity_lag_24": [latest_humidity] * len(future_dates)
})

# เพิ่มฟีเจอร์เชิงเวลา
future_data["hour"] = future_data["datetime"].dt.hour
future_data["dayofweek"] = future_data["datetime"].dt.dayofweek

# Recursive forecasting
for i in range(len(future_data)):
    pred = predict_model(model_humidity, data=future_data.iloc[i:i+1])
    predicted_value = pred.iloc[0]["prediction_label"]
    if i + 1 < len(future_data):
        future_data.at[i + 1, "humidity_lag_1"] = predicted_value
    if i + 3 < len(future_data):
        future_data.at[i + 3, "humidity_lag_3"] = predicted_value
    if i + 6 < len(future_data):
        future_data.at[i + 6, "humidity_lag_6"] = predicted_value
    if i + 24 < len(future_data):
        future_data.at[i + 24, "humidity_lag_24"] = predicted_value

# ทำนายค่า Humidity
predictions = predict_model(model_humidity, data=future_data)

# หาชื่อคอลัมน์การทำนายที่ถูกต้อง
prediction_column = [col for col in predictions.columns if "predict" in col.lower()]
if prediction_column:
    prediction_column = prediction_column[0]
    print(predictions[["datetime", prediction_column]])
else:
    print("ไม่พบคอลัมน์การทำนาย ลองแสดงผลทั้งหมด:")
    print(predictions.head())

future_data.to_csv("D:/project-term/pm_2.5/future_humidity_data.csv", index=False)

# บันทึกผลลัพธ์เป็นไฟล์ CSV โดยเลือกเฉพาะ datetime และ prediction_label
predictions[["datetime", "prediction_label"]].to_csv("D:/project-term/pm_2.5/predicted_humidity.csv", index=False)

print("บันทึกผลลัพธ์การพยากรณ์สำเร็จ! ✅")


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000253 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1070
[LightGBM] [Info] Number of data points in the train set: 18344, number of used features: 7
[LightGBM] [Info] Start training from score 48.890407


,Description,Value
0,Session id,42
1,Target,humidity
2,Target type,Regression
3,Original data shape,"(22930, 6)"
4,Transformed data shape,"(22930, 2)"
5,Transformed train set shape,"(18344, 2)"
6,Transformed test set shape,"(4586, 2)"
7,Numeric features,4
8,Date features,1
9,Preprocess,True


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,3.0829,16.8178,4.1009,0.7842,0.0869,0.0663
1,3.0645,16.7679,4.0949,0.7779,0.0871,0.0659
2,2.9555,15.1476,3.8920,0.8054,0.0808,0.0625
3,2.9929,16.1054,4.0132,0.7896,0.0853,0.0638
4,3.0264,15.5513,3.9435,0.7869,0.0830,0.0641
5,3.0170,15.0247,3.8762,0.7943,0.0814,0.0640
6,3.0529,16.5437,4.0674,0.7843,0.0854,0.0649
7,3.0148,15.8721,3.9840,0.7854,0.0842,0.0645
8,2.9745,15.0683,3.8818,0.7838,0.0828,0.0640


Transformation Pipeline and Model Successfully Saved


               datetime  prediction_label
0   2025-02-11 00:00:00         43.201765
1   2025-02-11 01:00:00         40.385086
2   2025-02-11 02:00:00         39.311582
3   2025-02-11 03:00:00         36.402889
4   2025-02-11 04:00:00         37.977754
..                  ...               ...
163 2025-02-17 19:00:00         46.761632
164 2025-02-17 20:00:00         47.969341
165 2025-02-17 21:00:00         46.430134
166 2025-02-17 22:00:00         46.940996
167 2025-02-17 23:00:00         46.292515

[168 rows x 2 columns]
บันทึกผลลัพธ์การพยากรณ์สำเร็จ! ✅


Temperature next 7 Days

In [ ]:
import pandas as pd
from pycaret.regression import *
from datetime import timedelta
import numpy as np

# โหลดไฟล์ CSV
file_path = "D:\project-term\pm_2.5\export-jsps013-1h.csv"
df = pd.read_csv(file_path)

# แปลง datetime เป็นชนิด datetime
df["datetime"] = pd.to_datetime(df["datetime"], dayfirst=True, errors="coerce")

# แปลง 'temperature' เป็นตัวเลข
df["temperature"] = pd.to_numeric(df["temperature"], errors="coerce")

# ลบคอลัมน์ 'pm_2_5' และ 'humidity' (หากยังอยู่ใน DataFrame)
for col in ["pm_2_5", "humidity"]:
    if col in df.columns:
        df.drop(columns=[col], inplace=True)

# สร้างสำเนา DataFrame เพื่อใช้ทำความสะอาดข้อมูล
df_cleaned = df.copy()

# ฟังก์ชัน Hybrid Imputation
def hybrid_imputation(series):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    non_outlier_values = series[(series >= lower_bound) & (series <= upper_bound)]
    median_value = non_outlier_values.median() if not non_outlier_values.empty else series.median()

    series = series.mask((series < lower_bound) | (series > upper_bound), np.nan)
    series = series.interpolate(method="linear")  
    series = series.fillna(median_value)  

    return series

# ใช้ Hybrid Imputation สำหรับ 'temperature'
df_cleaned["temperature"] = hybrid_imputation(df_cleaned["temperature"])

# สร้างฟีเจอร์ย้อนหลังของ temperature
for lag in [1, 3, 6, 24]:
    df_cleaned[f"temperature_lag_{lag}"] = df_cleaned["temperature"].shift(lag)

# ลบแถวที่มีค่า NaN หลังจากสร้าง Lag
df_cleaned.dropna(inplace=True)

# ตั้งค่า PyCaret และแบ่ง train 80% test 20%
exp = setup(df_cleaned, target="temperature", train_size=0.8, session_id=42, normalize=True, feature_selection=True)

# สร้างโมเดล Random Forest Regressor
model_temp = create_model("rf")

final_model = finalize_model(model_temp)
save_model(final_model, "D:/project-term/modelPM/model_temperature")

# ---- สร้างข้อมูลสำหรับพยากรณ์ล่วงหน้า 7 วัน (168 ชั่วโมง) ----
last_date = df_cleaned["datetime"].max()
future_dates = [last_date + timedelta(hours=i) for i in range(1, 169)]

# ใช้ค่าล่าสุดของ temperature
latest_temperature = df_cleaned["temperature"].iloc[-1]

future_data = pd.DataFrame({
    "datetime": future_dates,
    "temperature_lag_1": [latest_temperature] * len(future_dates),
    "temperature_lag_3": [latest_temperature] * len(future_dates),
    "temperature_lag_6": [latest_temperature] * len(future_dates),
    "temperature_lag_24": [latest_temperature] * len(future_dates)
})

# เพิ่มฟีเจอร์เชิงเวลา
future_data["hour"] = future_data["datetime"].dt.hour
future_data["dayofweek"] = future_data["datetime"].dt.dayofweek

# Recursive forecasting
for i in range(len(future_data)):
    pred = predict_model(model_temp, data=future_data.iloc[i:i+1])
    predicted_value = pred.iloc[0]["prediction_label"]
    if i + 1 < len(future_data):
        future_data.at[i + 1, "temperature_lag_1"] = predicted_value
    if i + 3 < len(future_data):
        future_data.at[i + 3, "temperature_lag_3"] = predicted_value
    if i + 6 < len(future_data):
        future_data.at[i + 6, "temperature_lag_6"] = predicted_value
    if i + 24 < len(future_data):
        future_data.at[i + 24, "temperature_lag_24"] = predicted_value

# ทำนายค่า Temperature
predictions = predict_model(model_temp, data=future_data)

# หาชื่อคอลัมน์การทำนายที่ถูกต้อง
prediction_column = [col for col in predictions.columns if "predict" in col.lower()]
if prediction_column:
    prediction_column = prediction_column[0]
    print(predictions[["datetime", prediction_column]])
else:
    print("ไม่พบคอลัมน์การทำนาย ลองแสดงผลทั้งหมด:")
    print(predictions.head())

future_data.to_csv("D:/project-term/pm_2.5/future_temperature_data.csv", index=False)

# บันทึกผลลัพธ์เป็นไฟล์ CSV โดยเลือกเฉพาะ datetime และ prediction_label
predictions[["datetime", "prediction_label"]].to_csv("D:/project-term/pm_2.5/predicted_temperature.csv", index=False)

print("บันทึกผลลัพธ์การพยากรณ์สำเร็จ! ✅")


,Description,Value
0,Session id,42
1,Target,temperature
2,Target type,Regression
3,Original data shape,"(22930, 6)"
4,Transformed data shape,"(22930, 2)"
5,Transformed train set shape,"(18344, 2)"
6,Transformed test set shape,"(4586, 2)"
7,Numeric features,4
8,Date features,1
9,Preprocess,True


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,1.0299,1.9676,1.4027,0.7905,0.0392,0.0298
1,1.0257,1.9016,1.3790,0.7833,0.0387,0.0298
2,1.0186,1.8749,1.3693,0.7958,0.0386,0.0297
3,1.0064,1.8364,1.3551,0.7977,0.0379,0.0291
4,1.0251,1.8561,1.3624,0.7862,0.0383,0.0298
5,1.0100,1.8140,1.3468,0.7984,0.0377,0.0292
6,1.0317,1.8831,1.3723,0.7908,0.0385,0.0300
7,1.0200,1.8340,1.3542,0.7869,0.0379,0.0296
8,1.0155,1.7787,1.3337,0.7875,0.0372,0.0294


Transformation Pipeline and Model Successfully Saved


               datetime  prediction_label
0   2025-02-11 00:00:00         33.955350
1   2025-02-11 01:00:00         34.784517
2   2025-02-11 02:00:00         34.018531
3   2025-02-11 03:00:00         33.487160
4   2025-02-11 04:00:00         34.364235
..                  ...               ...
163 2025-02-17 19:00:00         33.876294
164 2025-02-17 20:00:00         33.771180
165 2025-02-17 21:00:00         33.657665
166 2025-02-17 22:00:00         33.798971
167 2025-02-17 23:00:00         34.559749

[168 rows x 2 columns]
บันทึกผลลัพธ์การพยากรณ์สำเร็จ! ✅
